# Human-level control through deep reinforcement learning

## Introduction

The goal is to use a deep convolutional neural network to approximate the optimal action-value function:

<img src="pngs/dqn_1.png">

Reinforcement learning is known to be unstable or even to diverge when a nonlienar function approximator such as a NN is used to represent the action-value function. This instability has several causes: the correlations present in the sequence of observations, the fact that small updates to Q may significantly change the policy and therefore change the data distribution,
and the correlations between the action-values Q and the target values $\hat{T^*}Q$. The solution is to
1. use biologically inspired mechanism termed **experience relay** that randomizes over the data, thereby removing correlations in the observation sequence and smoothing over changes in the data distribution.
2. an iterative update that adjusts the action-values towards target values that are only periodically updated, thereby reducing correlations with the target.

While other stable methods exist for training neural networks in the
reinforcement learning setting, such as neural fitted Q-iteration, these methods involve the repeated training of networks of iterations. Consequently, these methods, are too inefficient to be used
successfully with large NNs. The algorithm parameterize an approximate value function $Q(s, a; \theta)$ using the deep convolutional neural network, in which $\theta_i$ are the parameters of the Q-network at iteration i. To perform experience replay, we store the agent's experiences $e_t = (s_t, a_t, r_t, S_{t+1})$ at each time-step t in a dataset $D_t = {e_1, ..., e_t}$.
During learning, we apply Q-learning updates, on samples (or mini-batches) of experience $(s, a, r, s^\prime) \approx U(D)$, drawn uniformly at random from the pool of stored samples. The Q-learning update at iteration i uses the following loss function:

<img src='pngs/dqn_2.png'>

This is similar as AVI with loss function $argmin_{Q \in F} E[|Q - \hat{T^{*}}Q_k|^2]$ (minimizing the unbiased sample $\hat{T^{*}}Q_k$ is the same as minimizing $T^{*}Q_k$). In which $\gamma$ is the discounted factor determining the agent's horizon, $\theta_i$ are the parameters of the Q-network at iteration i and $\theta^{-}_i$ are network parameters used to compute the target at iteration i. The target network parameters $\theta^{-}_i$ are only updated with the Q-network parameters $\theta_i$ every C steps and are held fixed between individual updates.

## Methods

### Model architecture

There are several possible ways of parameterizing Q using a
neural network. Because Q maps history–action pairs to scalar estimates of their
Q-value, the history and the action have been used as inputs to the neural network
by some previous approaches24,26. The main drawback of this type of architecture
is that a separate forward pass is required to compute the Q-value of each action,
resulting in a cost that scales linearly with the number of actions. (ie. *inputs are state action pair, output is a scalar value.*) Weinstead use an
architecture in which there is a separate output unit for each possible action, and
only the state representation is an input to the neural network. The outputs correspond
to the predicted Q-values of the individual actions for the input state.(ie. *input is state, outputs are Q values for each actions*) The main advantage of this type of architecture is the ability to comput Q-values for all possible actions in a given state with only a single forward pass through the network.

### Algorithm
We consider tasks in which an agent interacts with an environment, in this case, the Atari emulator, in a sequence of actions, observations and rewards, at each time-step the agent selects an action $a_t$ from the set of legal game actions, $A = \{1, ..., K\}$. The action is passed to the emulator and modifies its internal state and the game score. In general the environment may be stochastic. The emulator's internal state is not observed by the agent; instead the agent observes an image from the emulator, which is a vector of pixel values
representing the current screen. In addition, it receives a reward $r_t$ representing the change in game score. Note that, in general the game score may depend on the whole previous sequence of actions and ovservations; feedback about an action may only be received after many thousands of time-steps have elapsed.

Because the agent only observes the current screen, the task is partially observed and many emulator states are perceptually aliased (that is, it is impossible to fully
understand the current situation from only the current screen $x_t$). Therefore, sequences of actions and observations, $s_t = x_1, a_1, x_2, ..., a_{t-1}, x_t$ are input to the algorithm, which then learns game startegies depending upon these sequences. All sequences in the emulator are assumed to terminate in a finite number of time-steps. This formalism gives rise to a large but finite MDP in which each sequence is a distinct state. As a result, we can apply standard reinforcement learning methods for MDPs, simply by using the complete sequence $s_t$ as the state representation at time t.

The goal of the agent is to interact with the emulator by selecting actions in a way
that maximizes future rewards. We make the standard assumption that future rewards are discounted by a factor of $\gamma = 0.99$. The basic idea behind many reinforcement learning algorithms is to estimate the action-value function by using the bellman equation as an iterative update,

$Q_{t+1} (s, a) = T^{*} Q_{t} = E_{s^{\pi}} [\hat{T^*}Q_t(s, a) | s, a]$. In practice, this basic approach is impractical, because the action-value function is estimated separately for each sequence, without any generalization. Instead, it is common to use a function approximator to estimate the action-value function, $Q(s, a;\theta) \approx Q^{*} (x, a)$, this is usually a linear approximator, sometimes a non-linear function approximator is used instead such as NN. We refer to a NN function approximator with weights $\theta$ as a Q-network. A Q-network can be trained by adjusting the parameters $\theta_i$ at iteration i to reduce the MSE in the Bellman equation, where $T^{*} Q_{t}$ is substitute by an unbiased sample
$\hat{T^*}Q_t(s, a; \theta^{-}_i)$, using parameters $\theta^{-}_i$ from some previous iteration. This leads to a sequence of loss functions $L_i (\theta_i)$ that changes at each iteration i.

<img src="dqn_3.png">

The final term is the variance of the targets, which does not depend on the parameters $\theta_i$ that we are currently optimizing, and may therfore ignored (does not contribute to the gradient). Differentiating the loss function with respect to the weights we arrive at the following gradient:

<img src="dqn_4.png">

Rather than computing the full expectations in the above gradient, it is often
computationally expedient to optimize the loss function by stochastic gradient
descent. The familiar Q-learning algorithm can be recovered in this framework by updating the weights after every time setp, replacing the expectations using single samples, and setting $\theta^{-}_i = \theta_{i-1}$

Note that this algorithm is model-free, it solves the reinforcement learning task directly using samples from the emulator, without explicitly estimating the reward and transition dynamics P. It is also off-policy, it learns abou the greedy policy $a = argmax_{a^\prime} Q(s, a^{\prime}; \theta)$, while following a behaviour distribution that ensures adequate exploration of the state space. in practice, the behaviour distribution is often selected by an $\epsilon$-greedy policy that follows the greedy policy with probability $1 - \epsilon$ and
selects a random action with probability $\epsilon$

### Traning algorithm for DQN

The agent selects and executes actions
according to an e-greedy policy based on Q. Because using histories of arbitrary
length as inputs to a neural network can be difficult, our Q-function instead works
on a fixed length representation of histories produced by the function w described
above. The algorithm modifies standard online Q-learning in two ways to make it
suitable for training large neural networks without diverging.

First, we use a technique known as experience replay in which we store the agent's experiences at each time-step, $e_t = (s_t, a_t, r_t, s_{t+1})$, in a dataset $D_t = (e_1, ..., e_t)$, pooled over many episodes (where the end of an episode occurs when a terminal
state is reached) into a replay memory. During the inner loop of the algorithm, we apply Q-learning updates, or minibatch updates, to samples of experience, drawn at random from the pool of stored samples. This approach has several advantages over standard online Q-learning.
1. First, each step of experience is potentially used in amny weight updates, which allows for greater data efficiency.
2. Second, learning directly from consecutive samples is inefficient, owing to the strong correlations between the samples. Randomizing the samples breaks these correlations and therefore reduces the variance of the updates.
3. Third, when learning on-policy, the current parameters determine the next data sample that the parameters are trained on. For example, if the maximizing action is to move left then the traning samples will be dominated by samples from the left-hand side, if the maximizing action then switches to the right then the training distribution will also switch.

It is easy to see how unwanted feedback loops may arise, and the parameters could get stuck in a poor local minimum, or even diverge catastrophically. By using experience replay, the behaviour distirbution is averaged over many of its previous states, smoothing out learning and avoiding oscillations or divergence in the parameters. Note that, when learning by experience replay, it is necessary to learn off-policy because our current parameters are different to those used to generate the sample, which motivates the choice of Q-learning.

In practice, our algorithm only stores the last N experience tuples in the replay
memory, and samples uniformly at randomfromDwhen performing updates. This
approach is in some respects limited because the memory buffer does not differentiate
important transitions and always overwrites with recent transitions owing
to the finite memory size N. Similarly, the uniform sampling gives equal importance
to all transitions in the replay memory.Amore sophisticated sampling strategy
might emphasize transitions from which we can learn the most, similar to
prioritized sweeping

The second modification to online Q-learning aimed at further improving the
stability of our method with neural networks is to use a separate network for generating
the targets $y_i$ in the Q-learning update. More precisely, every C updates we clone the network Q to obtain a target network $\hat{Q}$ and use $\hat{Q}$ for generating the Q-learning targets $y_j$ for the following C updates to Q. This modeification makes the algorithm more stable compared to standard online Q-learning, where an update that increases $Q(s_t, a_t)$ often also increases $Q_{s_{t+1}, a}}, \forall a$, and hence also increase the target $y_j$, possibly leadning to oscillations or divergence of the policy.
Generating the targets using an older set of parameters adds a delay between the time an update to Q is made and the time the update affects the targets $y_j$, making divergence or oscillations much more unlikely.

we also found it helpful to clip the error term from the update $r + max_{a^\prime Q (s^\prime, a^\prime; \theta^{-}_{i}) - Q(s, a; \theta_i)$ to be between -1 and 1. Because the absolute value loss function |x| has a derivative of -1 for all negative values of x and a derivative of 1 for all positive values of x, clipping the squared error to be between -1 and 1 corresponding to using an absolute value loss function for errors outside of the -1, 1 interval. This form of error clipping further improved the stability of the algorithm.

<img src='dqn_5.png'>

